In [58]:
import copy
from dataclasses import dataclass, field
from enum import IntEnum
from typing import MutableSequence, Iterator
from typing import Sequence, TypeVar, Union, Optional

import clingo
import clingo.ast

from starlingo.util import typecheck, open_join_close


In [59]:
ForwardSymbol = TypeVar('ForwardSymbol', bound='Symbol')
ForwardSubSymbol = TypeVar('ForwardSubSymbol', bound='SubSymbol')
ForwardFunction = TypeVar('ForwardFunction', bound='Function')
ForwardUnaryOperator = TypeVar('ForwardUnaryOperator', bound='UnaryOperator')
ForwardBinaryOperator = TypeVar('ForwardBinaryOperator', bound='BinaryOperator')


class Symbol:

    def __neg__(self):
        return UnaryOperation(UnaryOperator.Minus, copy.deepcopy(self))

    def __add__(self, other):
        return BinaryOperation(copy.deepcopy(self), BinaryOperator.Plus, other)

    def is_function(self) -> bool:
        return isinstance(self, Function)

    def is_unary_operation(self) -> bool:
        return isinstance(self, UnaryOperation)

    def is_binary_operation(self) -> bool:
        return isinstance(self, BinaryOperation)

    def is_operation(self) -> bool:
        return self.is_unary_operation() or self.is_binary_operation()

    def is_variable(self) -> bool:
        return isinstance(self, Variable)

    def is_term(self) -> bool:
        return isinstance(self, Term)

    @classmethod
    def from_clingo_symbol(cls, symbol: clingo.Symbol) -> ForwardSymbol:
        if symbol.type is clingo.SymbolType.Function:
            name: str = symbol.name
            arguments = tuple(SubSymbol.from_clingo_symbol(argument) for argument in symbol.arguments)
            return Function(name, arguments)
        else:
            assert False, "Unknown clingo.SymbolType {}.".format(symbol.type)

    @classmethod
    def from_ast(cls, symbol: clingo.ast.AST) -> ForwardSymbol:
        if symbol.ast_type is clingo.ast.ASTType.Function:
            return Function.from_ast(symbol)
        elif symbol.ast_type is clingo.ast.ASTType.UnaryOperation:
            return UnaryOperation.from_ast(symbol)
        elif symbol.ast_type in (
                clingo.ast.ASTType.Variable, clingo.ast.ASTType.SymbolicTerm, clingo.ast.ASTType.BinaryOperation):
            return SubSymbol.from_ast(symbol)
        else:
            assert False, "Unknown clingo.ast.ASTType {} of clingo.ast.AST {}.".format(symbol.ast_type, symbol)


class SubSymbol(Symbol):

    @classmethod
    def from_clingo_symbol(cls, symbol: clingo.Symbol) -> ForwardSubSymbol:
        if symbol.type is clingo.SymbolType.Number:
            return Term(IntegerConstant(symbol.number))
        elif symbol.type is clingo.SymbolType.String:
            return Term(StringConstant(symbol.string))
        elif symbol.type is clingo.SymbolType.Function:
            f_symbol = Symbol.from_clingo_symbol(symbol)
            if symbol.negative:
                return UnaryOperation(UnaryOperator.Minus, f_symbol)
            return f_symbol
        else:
            assert False, "Unknown clingo.SymbolType {}.".format(symbol.type)

    @classmethod
    def from_ast(cls, symbol: clingo.ast.AST):
        if symbol.ast_type is clingo.ast.ASTType.Variable:
            return Variable.from_ast(symbol)
        elif symbol.ast_type is clingo.ast.ASTType.SymbolicTerm:
            return Term.from_ast(symbol)
        elif symbol.ast_type in (clingo.ast.ASTType.Function, clingo.ast.ASTType.UnaryOperation):
            return Symbol.from_ast(symbol)
        else:
            assert False, "Unknown clingo.ast.ASTType {} of clingo.ast.AST {}.".format(symbol.ast_type, symbol)


@dataclass(frozen=True, order=True)
class Variable(SubSymbol):
    name: str

    def __str__(self):
        return self.name

    @classmethod
    def from_ast(cls, variable: clingo.ast.AST):
        typecheck(variable, clingo.ast.ASTType.Variable, 'ast_type')
        return Variable(variable.name)


class Constant:
    pass


@dataclass(frozen=True, order=True)
class StringConstant(Constant):
    string: str = ""

    def __str__(self):
        return '"{}"'.format(self.string)


@dataclass(frozen=True, order=True)
class IntegerConstant(Constant):
    number: int = 0

    def __str__(self):
        return str(self.number)


@dataclass(frozen=True, order=True)
class Term(SubSymbol):
    constant: Constant = field(default=IntegerConstant())

    def __str__(self):
        return str(self.constant)

    @classmethod
    def from_ast(cls, term: clingo.ast.AST):
        typecheck(term, clingo.ast.ASTType.SymbolicTerm, 'ast_type')
        return SubSymbol.from_clingo_symbol(term.symbol)


class UnaryOperator(IntEnum):
    Minus = clingo.ast.UnaryOperator.Minus.value


@dataclass(frozen=True, order=True)
class UnaryOperation(Symbol):
    operator: UnaryOperator
    argument: SubSymbol

    def __str__(self) -> str:
        if self.operator is UnaryOperator.Minus:
            if self.argument.is_operation():
                return "-({})".format(self.argument)
            return "-{}".format(self.argument)
        else:
            assert False, "Unknown UnaryOperatorType {}.".format(self.operator)



class BinaryOperator(IntEnum):
    Plus = clingo.ast.BinaryOperator.Plus.value


@dataclass(frozen=True, order=True)
class BinaryOperation(SubSymbol):
    left: SubSymbol
    operator: BinaryOperator
    right: SubSymbol

    def __str__(self) -> str:
        left_str = str(self.left)
        if self.left.is_operation():
            left_str = '({})'.format(left_str)
        right_str = str(self.right)
        if self.right.is_operation():
            right_str = '({})'.format(right_str)

        if self.operator is BinaryOperator.Plus:
            return "{}+{}".format(left_str, right_str)


@dataclass(frozen=True, order=True)
class Function(Symbol):
    name: Optional[str] = None
    arguments: Sequence[SubSymbol] = ()

    @property
    def arity(self) -> int:
        return len(self.arguments)

    def __str__(self):
        if self.name is None and not self.arguments:
            return "()"
        elif self.name is not None and not self.arguments:
            return self.name
        elif self.name is None and self.arguments:
            return "({})".format(','.join(map(str, self.arguments)))
        else:
            return "{}({})".format(self.name, ','.join(map(str, self.arguments)))

    def match(self, name: Optional[str], arity: int = 0) -> bool:
        return name == self.name and arity == len(self.arguments)

    def match_signature(self, other: ForwardFunction) -> bool:
        return self.match(other.name, other.arity)

    @classmethod
    def from_ast(cls, fun: clingo.ast.AST):
        typecheck(fun, clingo.ast.ASTType.Function, 'ast_type')
        name = fun.name
        arguments = tuple(Symbol.from_ast(argument) for argument in fun.arguments)
        return Function(name, arguments)


In [60]:
ForwardAtom = TypeVar('ForwardAtom', bound='Atom')


@dataclass(frozen=True, order=True)
class Atom:
    symbol: Symbol = field(default_factory=Function)

    def __str__(self) -> str:
        return str(self.symbol)

    @staticmethod
    def from_clingo_symbol(symbol: clingo.Symbol) -> ForwardAtom:
        typecheck(symbol, clingo.SymbolType.Function, 'type')
        return Atom(Symbol.from_clingo_symbol(symbol))

    @classmethod
    def from_ast(cls, atom: clingo.ast.AST) -> ForwardAtom:
        typecheck(atom, clingo.ast.ASTType.SymbolicAtom, 'ast_type')
        symbol = Symbol.from_ast(atom.symbol)
        return Atom(symbol)


class ComparisonOperator(IntEnum):
    Equal = clingo.ast.ComparisonOperator.Equal.value
    GreaterEqual = clingo.ast.ComparisonOperator.GreaterEqual.value
    GreaterThan = clingo.ast.ComparisonOperator.GreaterThan.value
    LessEqual = clingo.ast.ComparisonOperator.LessEqual.value
    LessThan = clingo.ast.ComparisonOperator.LessThan.value
    NotEqual = clingo.ast.ComparisonOperator.NotEqual.value

    def __str__(self):
        if self is ComparisonOperator.Equal:
            op = '='
        elif self is ComparisonOperator.GreaterEqual:
            op = '>='
        elif self is ComparisonOperator.GreaterThan:
            op = '>'
        elif self is ComparisonOperator.LessEqual:
            op = '<='
        elif self is ComparisonOperator.LessThan:
            op = '<'
        else:
            assert self is ComparisonOperator.NotEqual, "Unknown ComparisonOperator {}".format(self)
            op = '!='
        return op


@dataclass(frozen=True, order=True)
class Comparison:
    left: SubSymbol = field(default_factory=Term)
    comparison: ComparisonOperator = field(default=ComparisonOperator.Equal)
    right: SubSymbol = field(default_factory=Term)

    def __str__(self):
        return "{}{}{}".format(self.left, self.comparison, self.right)

    @classmethod
    def from_ast(cls, comparison: clingo.ast.AST):
        typecheck(comparison, clingo.ast.ASTType.Comparison, 'ast_type')
        left = SubSymbol.from_ast(comparison.left)
        comparison_ = ComparisonOperator(comparison.comparison)
        right = SubSymbol.from_ast(comparison.right)
        return Comparison(left, comparison_, right)


In [61]:
ForwardLiteral = TypeVar('ForwardLiteral', bound='Literal')
ForwardBasicLiteral = TypeVar('ForwardBasicLiteral', bound='BasicLiteral')
ForwardConditionalLiteral = TypeVar('ForwardConditionalLiteral', bound='ConditionalLiteral')


class Sign(IntEnum):
    NoSign = 0
    DefaultNeg = 1


class Literal:
    def is_neg(self) -> bool:
        return NotImplemented

    def is_pos(self) -> bool:
        return NotImplemented

    @classmethod
    def from_ast(cls, literal: clingo.ast.AST) -> ForwardLiteral:
        if literal.ast_type is clingo.ast.ASTType.Literal:
            return BasicLiteral.from_ast(literal)
        elif literal.ast_type is clingo.ast.ASTType.ConditionalLiteral:
            return ConditionalLiteral.from_ast(literal)


@dataclass(frozen=True, order=True)
class BasicLiteral(Literal):
    atom: Atom = field(default_factory=Atom)
    sign: Sign = Sign.NoSign

    def is_neg(self) -> bool:
        return self.sign is Sign.DefaultNeg

    def is_pos(self) -> bool:
        return self.sign is Sign.NoSign

    def __str__(self):
        if self.sign is Sign.DefaultNeg:
            return "not {}".format(self.atom)
        return str(self.atom)

    def __abs__(self):
        return BasicLiteral(sign=Sign.NoSign, atom=copy.deepcopy(self.atom))

    def __neg__(self):
        return BasicLiteral(sign=Sign(self.sign ^ 1), atom=self.atom)

    def __invert__(self):
        return BasicLiteral(sign=Sign(self.sign ^ 1), atom=self.atom)

    @classmethod
    def from_ast(cls, literal: clingo.ast.AST) -> ForwardBasicLiteral:
        typecheck(literal, clingo.ast.ASTType.Literal, 'ast_type')
        sign = literal.sign
        if literal.atom.ast_type is clingo.ast.ASTType.SymbolicAtom:
            atom = Atom.from_ast(literal.atom)
        elif literal.atom.ast_type is clingo.ast.ASTType.Comparison:
            atom = Comparison.from_ast(literal.atom)
        else:
            assert False, "Unknown clingo.ast.ASTType {} of clingo.ast.AST {}.".format(literal.atom.ast_type, literal)
        return BasicLiteral(sign=Sign(sign), atom=atom)


@dataclass(frozen=True, order=True)
class ConditionalLiteral(Literal):
    literal: BasicLiteral = field(default_factory=BasicLiteral)
    condition: Sequence[Literal] = ()

    def is_pos(self) -> bool:
        return True

    def is_neg(self) -> bool:
        return False

    def __str__(self):
        if self.condition:
            return "{}: {}".format(self.literal, ', '.join(map(str, self.condition)))
        else:
            return str(self.literal)

    @classmethod
    def from_ast(cls, conditional_literal: clingo.ast.AST) -> ForwardConditionalLiteral:
        typecheck(conditional_literal, clingo.ast.ASTType.ConditionalLiteral, 'ast_type')
        basic_literal = BasicLiteral.from_ast(conditional_literal.literal)
        condition = tuple(Literal.from_ast(cond) for cond in conditional_literal.condition)
        return ConditionalLiteral(basic_literal, condition)


In [62]:
ForwardExternal = TypeVar('ForwardExternal', bound='External')
ForwardExternalType = TypeVar('ForwardExternalType', bound='ExternalType')
ForwardRule = TypeVar('ForwardRule', bound='Rule')
ForwardFact = TypeVar('ForwardFact', bound='Fact')
ForwardNormalRule = TypeVar('ForwardNormalRule', bound='NormalRule')
ForwardConstraint = TypeVar('ForwardConstraint', bound='Constraint')
ForwardChoiceRule = TypeVar('ForwardChoiceRule', bound='ChoiceRule')
ForwardDisjunctiveRule = TypeVar('ForwardDisjunctiveRule', bound='DisjunctiveRule')


class RuleLike:
    def is_rule(self) -> bool:
        return isinstance(self, Rule)

    def is_external(self) -> bool:
        return isinstance(self, External)

    @staticmethod
    def body_str(body):
        if any(isinstance(literal, ConditionalLiteral) for literal in body):
            return '; '.join(map(str, body))
        return ", ".join(map(str, body))


class Rule(RuleLike):

    def is_fact(self):
        if isinstance(self, Fact):
            return True
        elif self.is_normal_rule():
            assert isinstance(self, NormalRule) or isinstance(self, DisjunctiveRule)
            return not self.body
        else:
            return False

    def as_fact(self):
        if isinstance(self.head, Sequence):
            head = self.head[0]
        else:
            head = self.head
        return Fact(head)

    def is_normal_rule(self):
        if isinstance(self, NormalRule):
            return True
        if self.is_fact():
            return True
        elif isinstance(self, DisjunctiveRule):
            return len(self.head) == 1
        return False

    @classmethod
    def from_ast(cls, rule: clingo.ast.AST) -> ForwardRule:
        typecheck(rule, clingo.ast.ASTType.Rule, 'ast_type')
        if rule.head.ast_type is clingo.ast.ASTType.Literal:
            if rule.head.atom == clingo.ast.BooleanConstant(False):
                return Constraint.from_ast(rule)
            elif not rule.body:
                return Fact.from_ast(rule)
            else:
                return NormalRule.from_ast(rule)
        elif rule.head.ast_type is clingo.ast.ASTType.Aggregate:
            return ChoiceRule.from_ast(rule)
        elif rule.head.ast_type is clingo.ast.ASTType.Disjunction:
            return DisjunctiveRule.from_ast(rule)


@dataclass(frozen=True, order=True)
class Fact(Rule):
    head: Literal = field(default_factory=Literal)

    def __str__(self) -> str:
        return "{}.".format(self.head)

    @classmethod
    def from_ast(cls, fact: clingo.ast.AST) -> ForwardFact:
        typecheck(fact, clingo.ast.ASTType.Rule, 'ast_type')
        assert not fact.body, "clingo.ast.AST {} should not have a body.".format(fact)
        typecheck(fact.head, clingo.ast.ASTType.Literal, 'ast_type')
        head = Literal.from_ast(fact.head)
        return Fact(head)


@dataclass(frozen=True, order=True)
class NormalRule(Rule):
    head: Literal = field(default_factory=Literal)
    body: Sequence[Literal] = ()

    def __str__(self) -> str:
        return "{} :- {}.".format(self.head, Rule.body_str(self.body))

    @classmethod
    def from_ast(cls, normal: clingo.ast.AST) -> ForwardNormalRule:
        typecheck(normal.head, clingo.ast.ASTType.Literal, 'ast_type')
        assert normal.body, "clingo.ast.AST {} should have a non-empty body.".format(normal)
        head = Literal.from_ast(normal.head)
        body = tuple(Literal.from_ast(literal) for literal in normal.body)
        return NormalRule(head, body)


@dataclass(frozen=True, order=True)
class Constraint(Rule):
    body: Sequence[Literal] = ()

    def __str__(self) -> str:
        if self.body:
            return ":- {}.".format(Rule.body_str(self.body))
        else:
            return ":-."

    @classmethod
    def from_ast(cls, constraint: clingo.ast.AST) -> ForwardConstraint:
        assert constraint.head.atom == clingo.ast.BooleanConstant(
            False), "clingo.ast.AST {} should have head {}, but has {}.".format(constraint,
                                                                                clingo.ast.BooleanConstant(False),
                                                                                constraint.head.atom)
        body = tuple(Literal.from_ast(literal) for literal in constraint.body)
        return Constraint(body)


@dataclass(frozen=True, order=True)
class Guard:
    comparison: ComparisonOperator = field(default=ComparisonOperator.LessEqual)
    term: Union[Variable, Term] = field(default_factory=Term)


@dataclass(frozen=True, order=True)
class ChoiceRule(Rule):
    left_guard: Union[Guard, None] = None
    head: Sequence[ConditionalLiteral] = ()
    right_guard: Union[Guard, None] = None
    body: Sequence[Literal] = ()

    def __str__(self) -> str:
        if self.left_guard is None:
            left_guard_str = ''
        else:
            left_guard_str = "{} {} ".format(self.left_guard.term, self.left_guard.comparison)
        if self.right_guard is None:
            right_guard_str = ''
        else:
            right_guard_str = " {} {}".format(self.right_guard.comparison, self.right_guard.term)
        if self.body:
            body_str = " :- {}.".format(Rule.body_str(self.body))
        else:
            body_str = '.'
        return "{}{}{}{}{}{}".format(left_guard_str, '{', '; '.join(map(str, self.head)), '}', right_guard_str,
                                     body_str)

    @staticmethod
    def _get_guard(guard: Optional[clingo.ast.AST]) -> Guard:
        if guard is not None:
            typecheck(guard, clingo.ast.ASTType.AggregateGuard, 'ast_type')
            term = guard.term
            if term.ast_type is clingo.ast.ASTType.SymbolicTerm:
                term = Symbol.from_ast(term)
            elif term.ast_type is clingo.ast.ASTType.Variable:
                term = Variable.from_ast(term)
            else:
                assert False, "Unknown clingo.ast.ASTType {} of clingo.ast.AST {}.".format(term.ast_type, term)
            guard = Guard(ComparisonOperator(guard.comparison), term)
        return guard

    @classmethod
    def from_ast(cls, choice: clingo.ast.AST) -> ForwardChoiceRule:
        typecheck(choice.head, clingo.ast.ASTType.Aggregate, 'ast_type')
        elements = tuple(Literal.from_ast(element) for element in choice.head.elements)

        left_guard = ChoiceRule._get_guard(choice.head.left_guard)
        right_guard = ChoiceRule._get_guard(choice.head.right_guard)
        body = tuple(Literal.from_ast(literal) for literal in choice.body)
        return ChoiceRule(left_guard, elements, right_guard, body)


@dataclass(frozen=True, order=True)
class DisjunctiveRule(Rule):
    head: Sequence[ConditionalLiteral] = ()
    body: Sequence[Literal] = ()

    def __str__(self):
        if self.body:
            body_str = " :- {}.".format(Rule.body_str(self.body))
        else:
            body_str = '.'
        return "{}{}".format(' | '.join(map(str, self.head)), body_str)

    @classmethod
    def from_ast(cls, disjunctive_rule: clingo.ast.AST) -> ForwardDisjunctiveRule:
        typecheck(disjunctive_rule.head, clingo.ast.ASTType.Disjunction, 'ast_type')
        elements = tuple(ConditionalLiteral.from_ast(cond_literal) for cond_literal in disjunctive_rule.head.elements)
        body = tuple(Literal.from_ast(literal) for literal in disjunctive_rule.body)
        return DisjunctiveRule(elements, body)


class ExternalType(IntEnum):
    False_ = 0
    True_ = 1
    Free = 2
    Release = 3

    @staticmethod
    def from_truth_value(tv: clingo.TruthValue) -> ForwardExternalType:
        if tv is clingo.TruthValue.False_:
            return ExternalType.False_
        elif tv is clingo.TruthValue.True_:
            return ExternalType.True_
        elif tv is clingo.TruthValue.Free:
            return ExternalType.Free
        else:
            assert tv is clingo.TruthValue.Release
            return ExternalType.Release


@dataclass(frozen=True, order=True)
class External(RuleLike):
    atom: Atom = field(default_factory=Atom)
    body: Sequence[Literal] = ()
    external_type: ExternalType = ExternalType.False_

    def __str__(self):
        if not self.body:
            return "#external {}. [{}]".format(self.atom, self.external_type.name)
        return "#external {} : {}. [{}]".format(self.atom, ', '.join(map(str, self.body)), self.external_type.name)

    @classmethod
    def from_ast(cls, external: clingo.ast.AST) -> ForwardExternal:
        typecheck(external, clingo.ast.ASTType.External, 'ast_type')
        atom = Atom.from_ast(external.atom)
        body = tuple(Literal.from_ast(literal) for literal in external.body)
        external_type = ExternalType.from_truth_value(external.external_type)
        return External(atom, body, external_type)


In [63]:
@dataclass(frozen=True, order=True)
class Program:
    name: str = field(default='base')
    parameters: Sequence[Function] = field(default_factory=tuple)
    rules: Sequence[RuleLike] = field(default_factory=tuple)

    def custom_str(self, sep=' ', start='', end='') -> str:
        strs = (
            '#program {}{}.'.format(self.name, open_join_close(',', '(', ')', map(str, self.parameters))),
            *(map(str, self.rules)))
        return "{}{}{}".format(start, sep.join(strs), end)

    def __str__(self):
        return self.custom_str()


class _FromASTTransformer(clingo.ast.Transformer):

    def __init__(self):
        self.name: Optional[str] = None
        self.parameters: Optional[Sequence[Function]] = None
        self.program_rules: Optional[MutableSequence[RuleLike]] = None
        self.programs: MutableSequence[Program] = []

    def flush(self, name: Optional[str] = None, parameters: Sequence[clingo.ast.AST] = ()):
        if self.name is not None:
            self.programs.append(Program(self.name, self.parameters, self.program_rules))
        self.name = name
        self.parameters = tuple(Function.from_ast(parameter) for parameter in parameters)
        self.program_rules = []

    def visit_Program(self, program: clingo.ast.AST) -> clingo.ast.AST:
        self.flush(program.name, program.parameters)
        return program

    def visit_Rule(self, rule: clingo.ast.AST) -> clingo.ast.AST:
        if self.program_rules is not None:
            self.program_rules.append(Rule.from_ast(rule))
        return rule

    def visit_External(self, external: clingo.ast.AST) -> clingo.ast.AST:
        if self.program_rules is not None:
            self.program_rules.append(External.from_ast(external))
        return external


def from_string(program: str) -> Sequence[Program]:
    t = _FromASTTransformer()
    clingo.ast.parse_string(program, t.visit)
    t.flush()
    return t.programs


def evaluate_forwards(programs: Sequence[Program],
                      ctl: Optional[clingo.Control] = None,
                      parts=(('base', ()),),
                      report=False,
                      report_models=True,
                      report_result=True) -> Iterator[Sequence[Atom]]:
    if ctl is None:
        ctl = clingo.Control()
        ctl.configuration.solve.models = 0
    ctl.add('base', [], '\n'.join(map(lambda p: p.custom_str(sep='\n'), programs)))
    ctl.ground(parts)
    with ctl.solve(yield_=True) as solve_handle:
        models = 0
        for model in solve_handle:
            symbols = sorted(model.symbols(shown=True))
            if report and report_models:
                print("Answer {}:".format(model.number), end=' ')
                print("{",
                      '\n'.join(map(str, symbols)), "}", sep='\n')
            atoms = tuple(Atom.from_clingo_symbol(symbol) for symbol in symbols)
            models += 1
            yield atoms
        if report and report_result:
            solve_result = solve_handle.get()
            print(solve_result, end='')
            if solve_result.satisfiable:
                print(" {}{}".format(models, '' if solve_result.exhausted else '+'))


In [64]:
program = """

a :- b, d.
b :- d.
b :- irrelevant.
c :- a.
c :- irrelevant.
d.
d :- not a.

"""

In [65]:
programs = from_string(program)
program = programs[0]
print(program)

#program base. a :- b, d. b :- d. b :- irrelevant. c :- a. c :- irrelevant. d. d :- not a.


In [66]:
tuple(evaluate_forwards(programs, report=True));

Answer 1: {
a
b
c
d
}
SAT 1


<block>:4:6-16: info: atom does not occur in any rule head:
  irrelevant

<block>:6:6-16: info: atom does not occur in any rule head:
  irrelevant



In [67]:
_xclingo_trace = BasicLiteral(Atom(Function('_xclingo_trace')))
preprocessed_rules = []
labels = {}
for i, rule in enumerate(program.rules):
    if rule.is_rule():
        trace_rule_head = rule.head
        if rule.is_fact():
            trace_rule_body = (BasicLiteral(Atom(Function('_xclingo_trace'))),)
        else:
            trace_rule_body = (*rule.body, _xclingo_trace)
        preprocessed_rule = NormalRule(trace_rule_head, trace_rule_body)
        labels[rule] = i
        preprocessed_rules.append(
            Fact(BasicLiteral(Atom(Function('_xclingo_label', (Function('id'), Term(IntegerConstant(i)))))))
        )
        preprocessed_rules.append(
            preprocessed_rule
        )

print('\n'.join(map(str, preprocessed_rules)))
print('-' * 10)
for rule, id in labels.items():
    print("{}:".format(id), rule)

_xclingo_label(id,0).
a :- b, d, _xclingo_trace.
_xclingo_label(id,1).
b :- d, _xclingo_trace.
_xclingo_label(id,2).
b :- irrelevant, _xclingo_trace.
_xclingo_label(id,3).
c :- a, _xclingo_trace.
_xclingo_label(id,4).
c :- irrelevant, _xclingo_trace.
_xclingo_label(id,5).
d :- _xclingo_trace.
_xclingo_label(id,6).
d :- not a, _xclingo_trace.
----------
0: a :- b, d.
1: b :- d.
2: b :- irrelevant.
3: c :- a.
4: c :- irrelevant.
5: d.
6: d :- not a.


In [68]:
preprocessed = Program(rules=preprocessed_rules)
print(preprocessed.custom_str(sep='\n'))

#program base.
_xclingo_label(id,0).
a :- b, d, _xclingo_trace.
_xclingo_label(id,1).
b :- d, _xclingo_trace.
_xclingo_label(id,2).
b :- irrelevant, _xclingo_trace.
_xclingo_label(id,3).
c :- a, _xclingo_trace.
_xclingo_label(id,4).
c :- irrelevant, _xclingo_trace.
_xclingo_label(id,5).
d :- _xclingo_trace.
_xclingo_label(id,6).
d :- not a, _xclingo_trace.


In [69]:
translated_rules = []
for rule in preprocessed.rules:
    if rule.is_rule():
        if rule.is_normal_rule():
            if rule.head.atom.symbol.name.startswith('_xclingo'):
                translated_rules.append(rule)
                continue
            translated_body = []
            for literal in rule.body:
                symbol = literal.atom.symbol
                if symbol.name.startswith('_xclingo'):
                    translated_body.append(literal)
                else:
                    holds_literal = BasicLiteral(sign=literal.sign, atom=Atom(
                        Function('_xclingo_holds', (Function(symbol.name), Function(arguments=symbol.arguments)))))
                    translated_body.append(holds_literal)
            raw_rule = NormalRule(rule.head, rule.body[:-1])
            if not raw_rule.body:
                raw_rule = raw_rule.as_fact()
            label = labels[raw_rule]
            symbol = rule.head.atom.symbol
            translated_head = BasicLiteral(Atom(Function('_xclingo_fires', (
            Term(IntegerConstant(label)), symbol.name, Function(arguments=symbol.arguments)))))
            translated_rules.append(NormalRule(translated_head, translated_body))

translated = Program(rules=translated_rules)
print(translated.custom_str(sep='\n'))

#program base.
_xclingo_label(id,0).
_xclingo_fires(0,a,()) :- _xclingo_holds(b,()), _xclingo_holds(d,()), _xclingo_trace.
_xclingo_label(id,1).
_xclingo_fires(1,b,()) :- _xclingo_holds(d,()), _xclingo_trace.
_xclingo_label(id,2).
_xclingo_fires(2,b,()) :- _xclingo_holds(irrelevant,()), _xclingo_trace.
_xclingo_label(id,3).
_xclingo_fires(3,c,()) :- _xclingo_holds(a,()), _xclingo_trace.
_xclingo_label(id,4).
_xclingo_fires(4,c,()) :- _xclingo_holds(irrelevant,()), _xclingo_trace.
_xclingo_label(id,5).
_xclingo_fires(5,d,()) :- _xclingo_trace.
_xclingo_label(id,6).
_xclingo_fires(6,d,()) :- not _xclingo_holds(a,()), _xclingo_trace.


In [70]:
repositioned_rules = []
for rule in translated.rules:
    if rule.is_rule():
        if rule.is_normal_rule() and not rule.is_fact():
            if _xclingo_trace in rule.body:
                head = rule.head
                body = rule.body[:-1]

                fires_rule = NormalRule(head, body)
                support_head = BasicLiteral(Atom(Function('_xclingo_holds', head.atom.symbol.arguments[1:])))
                support_body = (head,)
                support_rule = NormalRule(support_head, support_body)
                trace_head = BasicLiteral(Atom(Function('_xclingo_trace', (
                head.atom.symbol.arguments[1], head.atom.symbol.arguments[2], Term(StringConstant(""))))))
                trace_body = support_body
                trace_rule = NormalRule(trace_head, trace_body)
                repositioned_rules.append(fires_rule)
                repositioned_rules.append(support_rule)
                repositioned_rules.append(trace_rule)

        else:
            repositioned_rules.append(rule)

repositioned = Program(rules=repositioned_rules)
print(repositioned.custom_str(sep='\n'))

#program base.
_xclingo_label(id,0).
_xclingo_fires(0,a,()) :- _xclingo_holds(b,()), _xclingo_holds(d,()).
_xclingo_holds(a,()) :- _xclingo_fires(0,a,()).
_xclingo_trace(a,(),"") :- _xclingo_fires(0,a,()).
_xclingo_label(id,1).
_xclingo_fires(1,b,()) :- _xclingo_holds(d,()).
_xclingo_holds(b,()) :- _xclingo_fires(1,b,()).
_xclingo_trace(b,(),"") :- _xclingo_fires(1,b,()).
_xclingo_label(id,2).
_xclingo_fires(2,b,()) :- _xclingo_holds(irrelevant,()).
_xclingo_holds(b,()) :- _xclingo_fires(2,b,()).
_xclingo_trace(b,(),"") :- _xclingo_fires(2,b,()).
_xclingo_label(id,3).
_xclingo_fires(3,c,()) :- _xclingo_holds(a,()).
_xclingo_holds(c,()) :- _xclingo_fires(3,c,()).
_xclingo_trace(c,(),"") :- _xclingo_fires(3,c,()).
_xclingo_label(id,4).
_xclingo_fires(4,c,()) :- _xclingo_holds(irrelevant,()).
_xclingo_holds(c,()) :- _xclingo_fires(4,c,()).
_xclingo_trace(c,(),"") :- _xclingo_fires(4,c,()).
_xclingo_label(id,5).
_xclingo_fires(5,d,()) :- .
_xclingo_holds(d,()) :- _xclingo_fires(5,d,()).
_x

In [71]:
models = tuple(evaluate_forwards([repositioned], report=True))

Answer 1: {
_xclingo_holds(a,())
_xclingo_holds(b,())
_xclingo_holds(c,())
_xclingo_holds(d,())
_xclingo_label(id,0)
_xclingo_label(id,1)
_xclingo_label(id,2)
_xclingo_label(id,3)
_xclingo_label(id,4)
_xclingo_label(id,5)
_xclingo_label(id,6)
_xclingo_fires(0,a,())
_xclingo_fires(1,b,())
_xclingo_fires(3,c,())
_xclingo_fires(5,d,())
_xclingo_trace(a,(),"")
_xclingo_trace(b,(),"")
_xclingo_trace(c,(),"")
_xclingo_trace(d,(),"")
}
SAT 1


<block>:11:27-56: info: atom does not occur in any rule head:
  _xclingo_holds(irrelevant,())

<block>:19:27-56: info: atom does not occur in any rule head:
  _xclingo_holds(irrelevant,())



In [72]:
model = models[0]

In [73]:
causes_table = {}
inverted_labels = {}
to_explain = []
for a in model:
    if a.symbol.name.startswith('_xclingo_holds'):
        explain = Function(a.symbol.arguments[0].name, a.symbol.arguments[1].arguments)
        fires = []
        for b in model:
            if b.symbol.name.startswith('_xclingo_fires'):
                if a.symbol.arguments[0] == b.symbol.arguments[1] and a.symbol.arguments[1] == b.symbol.arguments[2]:
                    rule_id = b.symbol.arguments[0].constant.number
                    if rule_id not in inverted_labels:
                        for rule, label in labels.items():
                            if label == rule_id:
                                inverted_labels[rule_id] = rule
                                break
                    fires.append(rule_id)

        causes_table[explain] = fires
    elif a.symbol.name.startswith('_xclingo_trace'):
        explain = Function(a.symbol.arguments[0].name, a.symbol.arguments[1].arguments)
        to_explain.append(explain)

for fun, causes in causes_table.items():
    print("{}:".format(fun), causes)

a: [0]
b: [1]
c: [3]
d: [5]


In [74]:
for label, rule in inverted_labels.items():
    print("{}: ".format(label), rule)

0:  a :- b, d.
1:  b :- d.
3:  c :- a.
5:  d.


In [75]:
print('\n'.join(map(str, to_explain)))

a
b
c
d


In [76]:
for explain in to_explain:
    print('*')
    stack = [(explain, 0)]
    while stack:
        holds, indent = stack.pop()
        print('    ' * indent + '|')
        print('    ' * indent + '+---+-', end=' ')
        print(holds)
        for cause in causes_table[holds]:
            rule = inverted_labels[cause]
            if rule.is_normal_rule() and not rule.is_fact():
                search = tuple((lit.atom.symbol,indent+1) for lit in rule.body if lit.is_pos())
                stack.extend(search)
    print()
    print('#'*24)
    print()


*
|
+---+- a
    |
    +---+- d
    |
    +---+- b
        |
        +---+- d

########################

*
|
+---+- b
    |
    +---+- d

########################

*
|
+---+- c
    |
    +---+- a
        |
        +---+- d
        |
        +---+- b
            |
            +---+- d

########################

*
|
+---+- d

########################

